In [116]:
import numpy as np
import cv2
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
import os
import pandas as pd
from skimage.measure import moments, moments_central, shannon_entropy
from sklearn.cluster import KMeans
import mahotas
from tqdm import tqdm
from scipy.stats import skew, kurtosis
import pywt
from pyefd import elliptic_fourier_descriptors, normalize_efd, reconstruct_contour

In [ ]:
def extract_contour_moments(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    contour_array = contour.reshape(-1, 2)
    
    # Calculate centroid
    M = cv2.moments(contour)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    
    distances = np.sqrt((contour_array[:, 0] - cx)**2 + (contour_array[:, 1] - cy)**2)
    
    mean_dist = np.mean(distances)
    std_dist = np.std(distances)
    min_dist = np.min(distances)
    max_dist = np.max(distances)
    
    dist_ratio = max_dist / (min_dist + 1e-6)
    
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    complexity = perimeter**2 / (4 * np.pi * area) if area > 0 else 0
    cdr = std_dist / mean_dist if mean_dist > 0 else 0.0
    return {
        'contour_std_dist': std_dist,
        'contour_min_dist': min_dist,
        'contour_max_dist': max_dist,
        'contour_dist_ratio': dist_ratio,
        'contour_complexity': complexity,
        'centroid_distance_ratio':cdr
    }


In [118]:
def lbp_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.uint8(image)  # Convert to uint8 before applying LBP
    # i_max = np.max(image)
    # i_min = np.min(image)
    # if i_max - i_min != 0:
    #     image = (image - i_min) / (i_max - i_min)
    lbp = local_binary_pattern(image, R=1, P=8, method="uniform")  
    hist, bins = np.histogram(lbp.ravel(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}

In [119]:
def texture_feature(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_img],[0],None, [256], [0,256])
    hist = hist/hist.sum()

    intensities = np.arange(256)
    mean = np.sum(intensities * hist.flatten())
    std = np.sqrt(np.sum((intensities-mean)**2 *hist.flatten()))
    uniformity = np.sum(hist.flatten()**2)
    third_moment = np.sum((intensities-mean)**3 * hist.flatten())
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }

In [120]:
def enhanced_color_feature(image): # 33 features
    def entropy(channel):
        return shannon_entropy(channel)
    def waveLet(channel):
        max_level = pywt.dwt_max_level(min(channel.shape), "db4")
        coeffs = pywt.wavedec2(channel, "db4", level=max_level)
        return np.mean(coeffs[0].ravel())
    
    # BGR
    B, G, R = cv2.split(image)
    mean_R, mean_G, mean_B = np.mean(R), np.mean(G), np.mean(B)
    # sqrt_R, sqrt_G, sqrt_B = np.sqrt(mean_R), np.sqrt(mean_G), np.sqrt(mean_B)
    std_R, std_G, std_B = np.std(R), np.std(G), np.std(B)
    skew_R, skew_G, skew_B = skew(R.flatten()), skew(G.flatten()), skew(B.flatten())
    kur_R, kur_G, kur_B = kurtosis(R.flatten()), kurtosis(G.flatten()), kurtosis(B.flatten())
    ent_R, ent_G, ent_B = entropy(R), entropy(G), entropy(B)
    wav_R, wav_G, wav_B = waveLet(R), waveLet(G), waveLet(B)
    
    # HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    # sqrt_h, sqrt_s, sqrt_v = np.sqrt(mean_h), np.sqrt(mean_s), np.sqrt(mean_v)
    skew_h, skew_s, skew_v = skew(h.flatten()), skew(s.flatten()), skew(v.flatten())
    kur_h, kur_s, kur_v = kurtosis(h.flatten()), kurtosis(s.flatten()), kurtosis(v.flatten())
    ent_h, ent_s, ent_v = entropy(h), entropy(s), entropy(v)
    wav_h, wav_s, wav_v = waveLet(h), waveLet(s), waveLet(v)
    
    # Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    mean_l, mean_a, mean_b = np.mean(l), np.mean(a), np.mean(b)
    std_l, std_a, std_b = np.std(l), np.std(a), np.std(b)
    # sqrt_l, sqrt_a, sqrt_b = np.sqrt(mean_l), np.sqrt(mean_a), np.sqrt(mean_b)
    skew_l, skew_a, skew_b = skew(l.flatten()), skew(a.flatten()), skew(b.flatten())
    kur_l, kur_a, kur_b = kurtosis(l.flatten()), kurtosis(a.flatten()), kurtosis(b.flatten())
    ent_l, ent_a, ent_b = entropy(l), entropy(a), entropy(b)
    wav_l, wav_a, wav_b = waveLet(l), waveLet(a), waveLet(b)
    
    # YCbCr
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb_img)
    mean_y, mean_cr, mean_cb = np.mean(y), np.mean(cr), np.mean(cb)
    std_y, std_cr, std_cb = np.std(y), np.std(cr), np.std(cb)
    # sqrt_y, sqrt_cr, sqrt_cb = np.sqrt(mean_y), np.sqrt(mean_cr), np.sqrt(mean_cb)
    skew_y, skew_cr, skew_cb = skew(y.flatten()), skew(cr.flatten()), skew(cb.flatten())
    kur_y, kur_cr, kur_cb = kurtosis(y.flatten()), kurtosis(cr.flatten()), kurtosis(cb.flatten())
    ent_y, ent_cr, ent_cb = entropy(y), entropy(cr), entropy(cb)
    wav_y, wav_cr, wav_cb = waveLet(y), waveLet(cr), waveLet(cb)
    
    # XYZ
    xyz_img = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    X, Y, Z = cv2.split(xyz_img)
    mean_X, mean_Y, mean_Z = np.mean(X), np.mean(Y), np.mean(Z)
    std_X, std_Y, std_Z = np.std(X), np.std(Y), np.std(Z)
    skew_X, skew_Y, skew_Z = skew(X.flatten()), skew(Y.flatten()), skew(Z.flatten())
    kur_X, kur_Y, kur_Z = kurtosis(X.flatten()), kurtosis(Y.flatten()), kurtosis(Z.flatten())
    ent_X, ent_Y, ent_Z = entropy(X), entropy(Y), entropy(Z)
    wav_X, wav_Y, wav_Z = waveLet(X), waveLet(Y), waveLet(Z)
    
    return {"mean_r": mean_R, "mean_g": mean_G, "mean_B": mean_B,
            # "sqrt_r": sqrt_R, "sqrt_g": sqrt_G, "sqrt_B": sqrt_B,
            "std_r": std_R, "std_g": std_G, "std_B": std_B,
            "skew_r": skew_R, "skew_g": skew_G, "skew_B": skew_B,
            "kur_r": kur_R, "kur_g": kur_G, "kur_B": kur_B,
            "ent_r": ent_R, "ent_g": ent_G, "ent_B": ent_B,
            "wav_r": wav_R, "wav_g": wav_G, "wav_B": wav_B,
            
            "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
            "std_h": std_h, "std_s": std_s, "std_v": std_v,
            # "sqrt_h": sqrt_h, "sqrt_s": sqrt_s, "sqrt_v": sqrt_v,
            "skew_h": skew_h, "skew_s": skew_s, "skew_v": skew_v,
            "kur_h": kur_h, "kur_s": kur_s, "kur_v": kur_v,
            "ent_h": ent_h, "ent_s": ent_s, "ent_v": ent_v,
            "wav_h": wav_h, "wav_s": wav_s, "wav_v": wav_v,
            
            "mean_l": mean_l, "mean_a": mean_a, "mean_b": mean_b,
            "std_l": std_l, "std_a": std_a, "std_b": std_b,
            # "sqrt_l": sqrt_l, "sqrt_a": sqrt_a, "sqrt_b": sqrt_b,
            "skew_l": skew_l, "skew_a": skew_a, "skew_b": skew_b,
            "kur_l": kur_l, "kur_a": kur_a, "kur_b": kur_b,
            "ent_l": ent_l, "ent_a": ent_a, "ent_b": ent_b,
            "wav_l": wav_l, "wav_a": wav_a, "wav_b": wav_b,
            
            "mean_y": mean_y, "mean_cb": mean_cb, "mean_cr": mean_cr,
            "std_y": std_y, "std_cb": std_cb, "std_cr": std_cr,
            # "sqrt_y": sqrt_y, "sqrt_cb": sqrt_cb, "sqrt_cr": sqrt_cr,
            "skew_y": skew_y, "skew_cb": skew_cb, "skew_cr": skew_cr,
            "kur_y": kur_y, "kur_cb": kur_cb, "kur_cr": kur_cr,
            "ent_y": ent_y, "ent_cb": ent_cb, "ent_cr": ent_cr,
            "wav_y": wav_y, "wav_cb": wav_cb, "wav_cr": wav_cr,
            
            "mean_x": mean_X, "mean_Y": mean_Y, "mean_z": mean_Z,
            "std_x": std_X, "std_Y": std_Y, "std_z": std_Z,
            "skew_x": skew_X, "skew_Y": skew_Y, "skew_z": skew_Z,
            "kur_x": kur_X, "kur_Y": kur_Y, "kur_z": kur_Z,
            "ent_x": ent_X, "ent_Y": ent_Y, "ent_z": ent_Z,
            "wav_x": wav_X, "wav_Y": wav_Y, "wav_z": wav_Z
            }

In [121]:
def zernike_feature(image, degree=8):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    mask = np.zeros(new_img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)  
    
    # Convert mask to binary for mahotas
    binary_mask = (mask > 0).astype(np.uint8)
    

    (x_center, y_center), radius = cv2.minEnclosingCircle(contour)

    radius = int(np.ceil(radius))
    

    x_center = int(x_center)
    y_center = int(y_center)
    x1 = max(x_center - radius, 0)
    y1 = max(y_center - radius, 0)
    x2 = x_center + radius
    y2 = y_center + radius
    
    cropped_mask = binary_mask[y1:y2, x1:x2]
    
    # padding
    h, w = cropped_mask.shape
    if h != w:
        size = max(h, w)
        square_mask = np.zeros((size, size), dtype=np.uint8)
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        square_mask[y_offset:y_offset+h, x_offset:x_offset+w] = cropped_mask
    else:
        square_mask = cropped_mask

    effective_radius = square_mask.shape[0] // 2
    zernike_moments = mahotas.features.zernike_moments(square_mask, effective_radius, degree)
    
    return {f'zernike_{i}': zernike_moments[i] for i in range(len(zernike_moments))}

In [ ]:
def test_feature(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    area = np.count_nonzero(new_img)
    
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(contour, True)
    
    x, y, w, h = cv2.boundingRect(contour)
    rect_perimeter = 2* (w+h)
    (x, y), radius = cv2.minEnclosingCircle(contour)
    #perimeter ratio
    pr = peri / rect_perimeter
    #sf6
    sf6 = peri / (2*np.sqrt(np.pi*area))
    ellipse = cv2.fitEllipse(contour)

    
    major_axis = max(ellipse[1])
    minor_axis = min(ellipse[1])
    ellipse_area = np.pi * major_axis * minor_axis / 4
    
    #elongation_factor
    ef = 1 - (minor_axis / major_axis)
    #elliptical deviation
    ed = area / ellipse_area
    
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    #convexity ratio
    cr = hull_perimeter / peri


    rect = cv2.minAreaRect(contour)
    width, height = rect[1]
    
    if width > height:
        width, height = height, width

    
    # Calculate area
    area_2 = cv2.contourArea(contour)
    min_rect_area = width * height
    
    # Calculate features
    min_rect_ratio = width / height if height > 0 else 0.0
    min_rect_efficiency = area_2 / min_rect_area if min_rect_area > 0 else 0.0
    
    #bending energy
    bending_energy = (peri**2) / area_2
    
    #circular_deviation
    circle_area = np.pi *(radius**2)
    circularity_deviation = area / circle_area
    
    dist = cv2.distanceTransform(new_img, cv2.DIST_L2, 5)
    
    # Calculate statistics of distance transform
    mask = new_img > 0
    if np.sum(mask) > 0:
        max_thickness = np.max(dist)
        mean_thickness = np.mean(dist[mask])

        relative_max = max_thickness / mean_thickness if mean_thickness > 0 else 0.0

    return {
        "perimeter_ratio": pr,
        "shape_factor_6": sf6,
        "elongation_factor": ef,
        "elliptical_deviation": ed,
        "convexity_ratio": cr,
        "min_rect_ratio": min_rect_ratio,
        "min_rect_efficiency": min_rect_efficiency,
        "bending_energy": bending_energy,
        "circularity_deviation": circularity_deviation,
        "relative_max_thickness": relative_max,
    }






In [125]:
def basic_feature(image): # 8 features
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(contour, True)
    x, y, w, h = cv2.boundingRect(contour)

    length = x + w
    width = y + h
    ratio = length / width
    
    ellipse = cv2.fitEllipse(contour)
    
    major_axis = max(ellipse[1])
    minor_axis = min(ellipse[1])
    
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    
    sf1 = major_axis / area
    sf2 = minor_axis / area
    sf3 = area / ((0.5 * major_axis)**2 * np.pi)
    sf4 = area / (0.5**2 * major_axis * minor_axis * np.pi)
    
    ed = np.sqrt(4 * area / np.pi)
    ar = major_axis / minor_axis
    roundness = (4 * area * np.pi) / peri**2
    Co = ed / major_axis
    solid = area / hull_area
    
    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis,
        "minor_axis_length": minor_axis,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
        "shape_factor_1": sf1,
        "shape_factor_2": sf2,
        "shape_factor_3": sf3,
        "shape_factor_4": sf4,
        # "shape_factor_6": sf6,
        "equivalent_diameter": ed,
        "aspect_ratio": ar,
        "perimeter": peri,
        "roundness": roundness,
        "compactness": Co,
        "solidity": solid
    }

In [126]:
from scipy.ndimage import convolve
def extract_edge_histogram_features(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernels = {
        "vertical": np.array([[-1,  2, -1],
                              [-1,  2, -1],
                              [-1,  2, -1]], dtype=np.float32),
        "horizontal": np.array([[-1, -1, -1],
                                [ 2,  2,  2],
                                [-1, -1, -1]], dtype=np.float32),
        "diag_45": np.array([[-1, -1,  2],
                             [-1,  2, -1],
                             [ 2, -1, -1]], dtype=np.float32),
        "diag_135": np.array([[ 2, -1, -1],
                              [-1,  2, -1],
                              [-1, -1,  2]], dtype=np.float32),
        "non_directional": np.array([[1,  1,  1],
                                     [1, -8,  1],
                                     [1,  1,  1]], dtype=np.float32)
    }
    
    features = {}
    total_energy = 0.0
    
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_{key}"] = energy
        total_energy += energy
    
    if total_energy > 0:
        for key in list(features.keys()):
            features[key] /= total_energy

    return features

In [ ]:
def extract_edge_histogram_features_2(image):
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    kernels = {

        # Sobel Operators
        "sobel_x": np.array([[-1, 0, 1],
                             [-2, 0, 2],
                             [-1, 0, 1]], dtype=np.float32),
        "sobel_y": np.array([[-1, -2, -1],
                             [ 0,  0,  0],
                             [ 1,  2,  1]], dtype=np.float32),
        # Prewitt Operators
        "prewitt_x": np.array([[-1, 0, 1],
                               [-1, 0, 1],
                               [-1, 0, 1]], dtype=np.float32),
        "prewitt_y": np.array([[-1, -1, -1],
                               [ 0,  0,  0],
                               [ 1,  1,  1]], dtype=np.float32),
        # Laplacian Operators
        "laplacian_4": np.array([[0,  1,  0],
                                 [1, -4,  1],
                                 [0,  1,  0]], dtype=np.float32),
        "laplacian_8": np.array([[1,  1,  1],
                                 [1, -8,  1],
                                 [1,  1,  1]], dtype=np.float32),
        # Roberts Cross Operators
        "roberts_x": np.array([[1, 0],
                               [0, -1]], dtype=np.float32),
        "roberts_y": np.array([[0, 1],
                               [-1, 0]], dtype=np.float32),
        # Scharr Operators
        "scharr_x": np.array([[-3,  0,  3],
                              [-10, 0, 10],
                              [-3,  0,  3]], dtype=np.float32),
        "scharr_y": np.array([[-3, -10, -3],
                              [ 0,   0,  0],
                              [ 3,  10,  3]], dtype=np.float32),
    }
    
    features = {}
    total_energy = 0.0
    
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_2_{key}"] = energy
        total_energy += energy
    
    if total_energy > 0:
        for key in features.keys():
            features[key] /= total_energy

    return features

In [ ]:
def compute_glcm_descriptor(image):
    if image is None:
        return None
    
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    distance = [3]  
    # angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]
    # properties = ['contrast', 'correlation', 'energy', 'homogeneity', 'dissimilarity']
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] 
    properties = ['contrast', 'correlation', 'energy', 'homogeneity','dissimilarity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [ ]:
def extract_all_features(image):    
    features = {}
    #10
    lbp_hist = lbp_feature(image) 
    features.update(lbp_hist) 
    #4
    texture_features = texture_feature(image)
    features.update(texture_features) 
    
    #25
    zernike_moment_features = zernike_feature(image)
    features.update(zernike_moment_features)
    
    contour_moment_features = extract_contour_moments(image)
    features.update(contour_moment_features)

    test_features = test_feature(image)
    features.update(test_features)
    #4

    #12
    color_features = enhanced_color_feature(image)
    features.update(color_features) 
    #5
    edge_histogram_features = extract_edge_histogram_features(image)
    features.update(edge_histogram_features)
    
    #
    edge_histogram_features_2 = extract_edge_histogram_features_2(image)
    features.update(edge_histogram_features_2)
    
    #8
    shape_features = basic_feature(image)
    if shape_features is not None:
        features.update(shape_features)
    #16
    glcm_features = compute_glcm_descriptor(image)
    if glcm_features is not None:
        for i, val in enumerate(glcm_features):
            features[f'GLCM_{i}'] = val


    return features


def process_directory(base_path):
    all_data = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                if 'negative' in root:
                    label = 0
                else:
                    label = 1
                image_paths.append((os.path.join(root, file), label))
    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue  

        features = extract_all_features(image)
        features["Label"] = label
 
        all_data.append(features)

    df = pd.DataFrame(all_data)

    return df

In [86]:

base_path = "/home/duyle/Rice_photos/rice_seed/TBR"

In [131]:
base_path = r'/home/duyle/Documents/AIL/rice_seed/TBR-45'
df = process_directory(base_path)

Processing Images:   0%|          | 0/2337 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/2337 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [112]:
df.to_csv(f'new_file_efd_TBR-45-2.csv',index=False)

In [113]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,GLCM_7,GLCM_8,GLCM_9,GLCM_10,GLCM_11,GLCM_12,GLCM_13,GLCM_14,GLCM_15,Label
0,504,1173,643,1930,3482,2954,1636,1344,2053,2141,...,0.876516,0.088325,0.080892,0.076210,0.078629,0.345646,0.285175,0.245843,0.269990,1
1,613,1170,715,1897,2936,2649,1432,1391,1864,2100,...,0.908837,0.084649,0.075170,0.071491,0.076803,0.313890,0.258703,0.232351,0.266753,1
2,380,864,507,1700,2991,2658,1330,1009,1592,1555,...,0.932737,0.086506,0.076586,0.073120,0.078300,0.339207,0.265794,0.241247,0.284220,1
3,410,913,523,1759,2885,2566,1512,1117,1787,1777,...,0.935297,0.093560,0.082427,0.079417,0.085350,0.354369,0.283624,0.262094,0.303779,1
4,572,1036,742,1779,2845,2644,1424,1245,1672,1874,...,0.879619,0.077745,0.070611,0.067074,0.069385,0.313767,0.260577,0.235049,0.256235,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,503,1025,615,1688,2944,2317,1169,1106,1480,1769,...,0.885183,0.068369,0.059321,0.054515,0.058399,0.295855,0.229553,0.192652,0.218205,0
2333,523,980,655,1762,2597,2679,1391,1212,1802,1876,...,0.897091,0.072664,0.065521,0.061782,0.065177,0.349146,0.294852,0.264624,0.293147,0
2334,773,1451,940,2362,3591,3213,1843,1673,2475,2679,...,0.883013,0.087901,0.080764,0.076535,0.080307,0.307358,0.258735,0.231358,0.251630,0
2335,411,991,579,1871,3389,2992,1495,1199,1898,1719,...,0.941727,0.072607,0.065884,0.062700,0.065771,0.355383,0.302969,0.266115,0.299286,0


In [22]:
types = ['BC-15','Xi-23']
for type in types:
    df = process_directory(f'/home/duyle/Documents/AIL/rice_seed/{type}')
    df.to_csv(f'new_file_{type}.csv',index=False)

Processing Images:   0%|          | 0/3759 [00:00<?, ?it/s]

Processing Images: 100%|██████████| 4579/4579 [06:53<00:00, 11.08it/s]


In [ ]:
types = ['BC-15','Huong_thom-1','Nep-87','Q-5','Thien_uu-8','Xi-23', 'TBR-36', 'TBR_45', 'TH3-5']
for type in types:
    df = process_directory(f'/home/duyle/Rice_photos/rice_seed/{type}')
    df.to_csv(f'all_with_zernike_pluscolor_enhanced_edge_{type}.csv',index=False)

Processing Images:   0%|          | 0/3422 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 2328/2328 [01:54<00:00, 20.30it/s]
Processing Images: 0it [00:00, ?it/s]
Processing Images:   0%|          | 0/2074 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 2074/2074 [01:48<00:00, 19.07it/s]


In [67]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,GLCM_7,GLCM_8,GLCM_9,GLCM_10,GLCM_11,GLCM_12,GLCM_13,GLCM_14,GLCM_15,Label
0,249,667,355,1761,2894,2749,1194,921,1615,1329,...,0.919322,0.087261,0.076275,0.072825,0.076559,0.356618,0.279062,0.248866,0.286045,1
1,344,833,411,1739,2987,2735,1278,798,1460,1489,...,0.899277,0.077756,0.067537,0.063615,0.068074,0.340384,0.262537,0.228819,0.257738,1
2,485,1047,672,2118,3141,3007,1522,1202,2016,1966,...,0.910777,0.086017,0.078133,0.074423,0.078500,0.344613,0.284887,0.255701,0.285575,1
3,567,997,707,1824,2960,2570,1392,1184,1621,1873,...,0.903198,0.071537,0.063342,0.061175,0.064125,0.305813,0.246045,0.218532,0.250488,1
4,541,1004,560,1536,2739,2108,1078,1067,1325,2058,...,0.859189,0.064338,0.055301,0.052004,0.055064,0.291228,0.230163,0.199191,0.223099,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3417,549,1109,800,2156,3089,2942,1720,1276,2028,2061,...,0.917721,0.089944,0.083424,0.080165,0.083763,0.329930,0.285341,0.260048,0.288865,0
3418,325,909,524,2151,3341,3508,1509,1149,2382,2454,...,0.939424,0.110095,0.100542,0.097592,0.102480,0.380486,0.310400,0.287083,0.329649,0
3419,449,1067,592,2365,3376,3380,1773,1366,2238,2114,...,0.932853,0.077291,0.070693,0.067480,0.071971,0.376489,0.319709,0.289111,0.330268,0
3420,334,725,447,1605,3184,2582,1185,817,1292,1401,...,0.927336,0.070488,0.060200,0.055431,0.060148,0.334753,0.261893,0.232836,0.263993,0
